DISSENT : Decision-making In Social Systems via Emergent Network Theory


In [ ]:
# STEP 1 :- Define voter groups -> This can be modified to include more diverse groups
voter_groups = {
    "Young men without college degree": {
        "weight": 0.25,
        "turnout": 0.35,
        "prefrences": [0.9, 0.5, 0.7]
    },
    "Young men with college degree": {
        "weight": 0.20,
        "turnout": 0.45,
        "prefrences": [0.9, 0.7, 0.7]
    },
    "Young women": {
        "weight": 0.25,
        "turnout": 0.55,
        "prefrences": [0.9, 0.7, 0.8]
    },
    "Asian youth": {
        "weight": 0.15,
        "turnout": 0.5,
        "prefrences": [0.9, 0.8, 0.7]
    },
    "Black youth": {
        "weight": 0.15,
        "turnout": 0.5,
        "prefrences": [0.9, 0.2, 0.7]
    }
}


In [ ]:
# STEP 2 :- Define policy bundles (3-parameter: Economic, Climate, Healthcare)
# A point to be noted is that the 3 parameters(or issues) is an assumption by me based on the Tufts Research. Read the documentation to know more.
# You could choose different parameters and policy bundles based on target community

policy_bundles = {
    "National Gun Ban": [0.3, 0.2, 0.4],
    "Pro-Choice Constitutional Right": [0.5, 0.2, 0.7],
    "Green New Deal": [0.1, 0.9, 0.6],
    "Ban Gender-Affirming Care": [0.2, 0.2, 0.1],
    "Universal Reproductive Health": [0.3, 0.2, 0.9],
    "Religious Freedom Act": [0.7, 0.2, 0.5]
}

In [ ]:
influence_matrix = [
    [1.0, 0.1, 0.2, 0.1, 0.1],
    [0.1, 1.0, 0.2, 0.2, 0.1],
    [0.2, 0.2, 1.0, 0.2, 0.1],
    [0.1, 0.2, 0.2, 1.0, 0.2],
    [0.1, 0.1, 0.2, 0.2, 1.0]
]

# Applying  influence function
def apply_influence(pref_list, influence_matrix, alpha=0.1):
    influenced = []
    for i in range(len(pref_list)):
        new_pref = [0.0, 0.0, 0.0]
        total_weight = 0
        for j in range(len(pref_list)):
            w = influence_matrix[i][j]
            total_weight += w
            for k in range(3):
                new_pref[k] += w * pref_list[j][k]
        avg_pref = [p / total_weight for p in new_pref]
        mixed_pref = [
            min(1, max(0, (1 - alpha) * pref_list[i][k] + alpha * avg_pref[k]))
            for k in range(3)
        ]
        influenced.append(mixed_pref)
    return influenced

In [ ]:
def dot_product(list1,list2):
  sum=0
  for i in range(len(list1)):
    sum+=list1[i]*list2[i]
  return round(sum,2)

In [ ]:
import random
random.seed(1)

# Simulating
winner_counts={}
for k in policy_bundles:
  winner_counts[k]=0

for sim in range(100000):
    l = []
    pref = []
    for i, j in voter_groups.items():
        l.append(j["turnout"])
        pref.append(j["prefrences"])

    noisy_turnout = [min(1, max(0, i * random.normalvariate(1, 0.4))) for i in l]

    noisy_pref = []
    for i in pref:
        l_pref1 = []
        for j in i:
            l_pref1.append(min(1, max(0, j * random.normalvariate(1, 0.4))))
        noisy_pref.append(l_pref1)

    # Applying emergent influence
    noisy_pref = apply_influence(noisy_pref, influence_matrix, alpha=0.1)

    # Scoring policies and finding the winner
    dict3 = {}
    for j, k in policy_bundles.items():
        list1 = []
        count = 0
        for i in voter_groups.values():
            list1.append(round(dot_product(noisy_pref[count], k) * i["weight"] * noisy_turnout[count], 3))
            count += 1
        dict3[j] = list1


    dict6 = {i: round(sum(j), 3) for i, j in dict3.items()}
    winner = max(dict6, key=dict6.get)
    winner_counts[winner] += 1

# Le us print the final result
print("\nWinner counts after 100,000 simulations:")
for k, v in sorted(winner_counts.items(), key=lambda x: -x[1]):
    print(f"{k}: {v}")


In [ ]:
import random
# Now let us change values of alpha to understand the effect of echo chambers


alpha_values = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
SIMULATIONS = 100000

random.seed(100)

def run_simulation(alpha=0.0, noise=0.4, simulations=SIMULATIONS):
    winner_counts = {}
    for k in policy_bundles:
      winner_counts[k]=0

    for a in range(simulations):
        l, pref = [], []
        for j in voter_groups.values():
            l.append(j["turnout"])
            pref.append(j["prefrences"])

        noisy_turnout = [min(1, max(0, i * random.normalvariate(1, noise))) for i in l]

        noisy_pref = []
        for i in pref:
            l_pref1 = []
            for j in i:
                l_pref1.append(min(1, max(0, j * random.normalvariate(1, noise))))
            noisy_pref.append(l_pref1)

        # Here we take alpha > O because at alpha = 0 is the baseline condition which is when voter vote independently.
        if alpha > 0:
            noisy_pref = apply_influence(noisy_pref, influence_matrix, alpha=alpha)

        dict3 = {}
        for j, k in policy_bundles.items():
            list1 = []
            count = 0
            for i in voter_groups.values():
                score = dot_product(noisy_pref[count], k) * i["weight"] * noisy_turnout[count]
                list1.append(round(score, 3))
                count += 1
            dict3[j] = list1

        dict6 = {i: round(sum(j), 3) for i, j in dict3.items()}
        winner = max(dict6, key=dict6.get)
        winner_counts[winner] += 1

    return winner_counts

# ENT Results
ent_results = {}
for alpha in alpha_values:
    print(f"Running ENT simulation with alpha={alpha}")
    ent_results[alpha] = run_simulation(alpha)

for alpha, results in ent_results.items():
    print(f"\nENT Alpha = {alpha:.1f}")
    for policy, count in results.items():
        print(f"  {policy}: {count}")
